In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import optuna
import mlflow
import mlflow.pytorch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


c:\Users\moham\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_csv("../data/processed_data.csv")

X = df.drop("fail", axis=1).values
y = df["fail"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)    

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [22]:
class myNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout):
        super(myNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


In [26]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())  # should now be True


False


In [28]:
def objective(trial):
 
    params = {
        "hidden_dim": trial.suggest_int(   "hidden_dim", 32, 128 ),
        "lr":         trial.suggest_float( "lr", 1e-4, 1e-2, log=True ),
        "dropout":    trial.suggest_float( "dropout", 0.1, 0.5 ),
        "epochs":     trial.suggest_int(   "epochs", 5, 15 )
    }

    model     = myNN(X_train.shape[1], params["hidden_dim"], params["dropout"])
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])

    with mlflow.start_run(nested=True):
        mlflow.log_params(params)

        for epoch in range(params["epochs"]):
            model.train()
            running_train_loss = 0
            all_train_preds = []
            all_train_labels = []

            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                batch_y = batch_y.float()  

                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

                running_train_loss += loss.item()
                preds = (outputs > 0.5).int()
                all_train_preds.append(preds)
                all_train_labels.append(batch_y.int())

            all_train_preds = torch.cat(all_train_preds).squeeze()
            all_train_labels = torch.cat(all_train_labels).squeeze()
            train_acc = accuracy_score(all_train_labels, all_train_preds)
            avg_train_loss = running_train_loss / len(train_loader)

            model.eval()
            with torch.no_grad():
                val_outputs = model(X_test_tensor)
                val_loss = criterion(val_outputs, y_test_tensor)
                val_pred = (val_outputs > 0.5).int().squeeze()
                val_acc = accuracy_score(y_test_tensor.int(), val_pred)

            
            mlflow.log_metric("train_loss", avg_train_loss, step=epoch)
            mlflow.log_metric("train_accuracy", train_acc, step=epoch)
            mlflow.log_metric("val_loss", val_loss.item(), step=epoch)
            mlflow.log_metric("val_accuracy", val_acc, step=epoch)
            mlflow.pytorch.log_model(model,"model")
            

        return val_loss  


In [29]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Machine_Failure_PyTorch")

study = optuna.create_study(direction="minimize")
with mlflow.start_run(run_name="Optuna_PyTorch"):
    study.optimize(objective, n_trials=15)

print("Best trial:", study.best_value)
print("Best params:", study.best_params)


[I 2025-05-20 22:18:01,899] A new study created in memory with name: no-name-45bb353e-c27e-4ba4-8442-d27797803d6f


2025/05/20 22:18:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 22:18:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 22:18:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 22:18:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 22:18:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 22:18:46 

🏃 View run abundant-newt-576 at: http://127.0.0.1:5000/#/experiments/820496218632962888/runs/01ecf338190548798d54671e66f473f6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/820496218632962888


2025/05/20 22:19:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 22:19:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 22:20:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 22:20:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 22:20:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 22:20:28 

🏃 View run burly-shark-927 at: http://127.0.0.1:5000/#/experiments/820496218632962888/runs/250dc54e490f465b8ff8d29ba69c0989
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/820496218632962888


...

🏃 View run wise-fly-779 at: http://127.0.0.1:5000/#/experiments/820496218632962888/runs/159dc6b0c7e840b2b880d12ed9bb724d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/820496218632962888


2025/05/20 22:35:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 22:35:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 22:35:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 22:35:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 22:35:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 22:35:44 

🏃 View run hilarious-stoat-637 at: http://127.0.0.1:5000/#/experiments/820496218632962888/runs/bd258e53667d452db94b04568d260157
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/820496218632962888


2025/05/20 22:36:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 22:36:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 22:36:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 22:36:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 22:36:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-05-20 22:36:

🏃 View run painted-dove-177 at: http://127.0.0.1:5000/#/experiments/820496218632962888/runs/44e70b5085b84114a4d7523cbaddc6a4
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/820496218632962888
🏃 View run Optuna_PyTorch at: http://127.0.0.1:5000/#/experiments/820496218632962888/runs/8687945d17af43e09f006b1860694171
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/820496218632962888
Best trial: 0.2937333285808563
Best params: {'hidden_dim': 71, 'lr': 0.0016109145194270799, 'dropout': 0.43305797500170606, 'epochs': 5}


In [31]:
best_params = study.best_params
final_model = myNN(X_train.shape[1], best_params["hidden_dim"], best_params["dropout"])
final_model = final_model.to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(final_model.parameters(), lr=best_params["lr"])

for epoch in range(best_params["epochs"]):
    final_model.train()
    for batch_X, batch_y in train_loader:
       
        optimizer.zero_grad()
        outputs = final_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

final_model.eval()
with torch.no_grad():
    train_outputs = final_model(X_train_tensor.to(device))
    train_preds = (train_outputs > 0.5).int().squeeze()
    train_acc = accuracy_score(y_train_tensor.cpu().int(), train_preds.cpu())

with torch.no_grad():
    test_outputs = final_model(X_test_tensor)
    test_preds = (test_outputs > 0.5).int().squeeze()
    test_acc = accuracy_score(y_test_tensor.cpu().int(), test_preds.cpu())

with mlflow.start_run(run_name="Final_PyTorch_Model"):
    mlflow.log_params(best_params)
    mlflow.log_metric("train_accuracy", train_acc)
    mlflow.log_metric("test_accuracy", test_acc)
    mlflow.pytorch.log_model(final_model, "model")

    print(f"Final Train Accuracy: {train_acc:.4f}")
    print(f"Final Test Accuracy: {test_acc:.4f}")
torch.save(final_model.state_dict(), "model_weights.pt")


2025/05/20 23:34:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Final Train Accuracy: 0.9285
Final Test Accuracy: 0.8783
🏃 View run Final_PyTorch_Model at: http://127.0.0.1:5000/#/experiments/820496218632962888/runs/77f2510b94304b7584da6896cf8eb187
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/820496218632962888
